In [13]:
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
import requests

# Load environment variables
load_dotenv()

# Replace 'YOUR_ACCESS_TOKEN' and 'YOUR_BITLINK' with your actual values
access_token = os.environ.get("BITLY2")
bitlink_id = "bit.ly/sema_01"  # Replace with your Bitlink ID (just the part after 'bit.ly/')

def get_bitlink_total_clicks(access_token, bitlink_id):
    # Prepare the headers for the request
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    recent_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S+0000')
    # Set the parameters for the metrics
    params = {
        'unit': 'month',
        'units': '1',
        'unit_reference': recent_date
    }
    url = f'https://api-ssl.bitly.com/v4/bitlinks/{bitlink_id}/clicks'
    # Prepare the API request
    response = requests.get(url, headers=headers, params=params)

    # Check the response status code and extract the click metrics if successful
    if response.status_code == 200:
        response_data = response.json()
        if 'link_clicks' in response_data and response_data['link_clicks']:
            total_clicks = sum(item.get('clicks', 0) for item in response_data['link_clicks'])
            return total_clicks
        else:
            print("No click data available.")
    else:
        print(f"Failed to retrieve click metrics. Status code: {response.status_code}")
        print("Message:", response.text)

    return 0

# Execute the function with your access token and Bitlink ID
total_clicks = get_bitlink_total_clicks(access_token, bitlink_id)
print(f"Total link clicks: {total_clicks}")

Total link clicks: 3


In [8]:
import psycopg2
import os

def connection():
    """Creates and returns a new database connection."""
    try:
        conn = psycopg2.connect(
            user=os.environ["MY_INTEGRATION_USER"],
            password=os.environ["MY_INTEGRATION_PASSWORD"],
            host=os.environ["MY_INTEGRATION_HOST"],
            port=os.environ["MY_INTEGRATION_PORT"],
            database=os.environ["MY_INTEGRATION_DATABASE"]
        )
        
        # Test the connection
        with conn.cursor() as cursor:
            cursor.execute("SELECT version();")
            record = cursor.fetchone()
        
        return conn  # Return the connection object if successful

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to database", error)
        return None  # Return None if connection was not successful

In [9]:
!pip install art


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
import time
from IPython.display import clear_output
from rich.console import Console
from rich.table import Table
from IPython.display import clear_output
from rich.panel import Panel
from rich.layout import Layout
from art import *

def display_all_jobs():
    # Connect to the database
    conn = connection()  # Adjust with your actual credentials
    c = conn.cursor()
    console = Console()
    while True:  # This will start the infinite loop
        try:
            # Fetch the status of all jobs that are currently running
            c.execute("SELECT job_id, job_status, query, arxiv_links_count FROM jobs WHERE job_status = 'running'")
            jobs = c.fetchall()
            # Get the total clicks for the Bitlink
            total_clicks = get_bitlink_total_clicks(access_token, bitlink_id)
            # Initialize a Rich table
            table = Table(show_header=True, title="Current Running Jobs", expand=True)
            table.add_column("Job ID", style="magenta")
            table.add_column("Query", style="green")
            table.add_column("Status", style="cyan")
            table.add_column("Last Log", justify="left")
            table.add_column("arXiv Links Count", style="yellow")  # New column for arXiv links count

            for job in jobs:
                job_id, job_status, query, arxiv_links_count = job  # Unpack job details with arXiv links count

                # Attempt to read the last line from the corresponding log file
                log_file_path = os.path.join(os.getcwd(), 'logs', f"{job_id}.log")
                last_log = "No log file found."
                if os.path.exists(log_file_path):
                    with open(log_file_path, 'r') as log_file:
                        lines = log_file.readlines()
                        if lines:
                            # Assume log format "[timestamp] - message" and remove timestamp
                            last_log = lines[-1].partition('] - ')[2].strip() if '] - ' in lines[-1] else lines[-1].strip()

                # Add job details to the table
                table.add_row(str(job_id), query, job_status, last_log, str(arxiv_links_count))

            # Create a layout with the total clicks and the table
            layout = Layout()
            layout.split(
                Layout(name="header", size=10),
                Layout(name="main", ratio=1),
            )
            # Convert total_clicks to ASCII and store it in a string
            ascii_total_clicks = text2art(str(total_clicks))  # Convert number to ASCII art
            ascii_art_10 = text2art("10")
            # Update the layout with ASCII art
            ascii_title_and_clicks = "Total number of QR-clicks\n" + text2art(str(total_clicks))
            layout["header"].update(Panel(ascii_title_and_clicks, style="blue"))
            layout["main"].update(Panel(table))

            clear_output(wait=True)
            console.print(layout)
            time.sleep(1)  # Adjust the sleep time as needed

        except Exception as e:
            console.print(f"[red]Error while fetching data: {e}[/red]")
            break  # Break out of the loop in case of an error

    # Once the loop is exited, close database resources
    if conn:
        c.close()
        conn.close()

# Usage example
display_all_jobs()


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Total number of QR-clicks                                                                                       │
│  _____                                                                                                          │
│ |___  |                                                                                                         │
│    / /                                                                                                          │
│   / /                                                                                                           │
│  /_/                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                              Current Running Jobs                                               │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Job ID          ┃ Query        ┃ Status         ┃ Last Log           ┃ arXiv Links Count                    ┃ │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ └─────────────────┴──────────────┴────────────────┴────────────────────┴──────────────────────────────────────┘ │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                       

# Scan me to use SEMA - Semantic search agent for arXiv (we collect all your data so AI can learn faster)

<img src="image-20240317-094547.png" width="" align="" />

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d52007a-f237-4857-b1f1-3ccb95216ee4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>